# Figures for the checkerboard function

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=1.5)

import numpy as np
import nshap
import os

### Visualization in 2d

In [ ]:
from paperutil import checkerboard_function

d = 2
num_points = 1000
f = checkerboard_function(2, 4)
X = np.random.uniform(0, 1, size=(num_points, d))
Y = f(X)

sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=Y)
plt.show()

### Load the pre-computed n-Shpaley Values

In [ ]:
results = {}
for degree in range(2, 11):
    results[degree] = {}
    for num_samples in [100, 1000, 10000, 100000, 1000000]:
        results[degree][num_samples] = []
        for replication in range(10):
            fname = f'../../results/n_shapley_values/checkerboard/{degree}d_checkerboard_{num_samples}_samples_replication_{replication}.JSON'
            if os.path.exists(fname):
                results[degree][num_samples].append( nshap.load(fname) )

### Correctly estimated fractions

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 2},  font_scale=2)

plt.figure(figsize=(12.5, 8))
for checkerboard_dim in range(2, 11):
    y = []
    for num_samples in [100, 1000, 10000, 100000, 1000000]:
        yy = []
        for x in results[checkerboard_dim][num_samples]:
            true_order = np.sum([np.abs(v) for k, v in x.items() if len(k) == checkerboard_dim]) 
            all_orders = np.sum([np.abs(v) for k, v in x.items()])
            yy.append(true_order / all_orders)
        y.append(np.mean(yy))
    sns.scatterplot(x=[1,2,3,4,5], y=y, color=sns.color_palette("tab10")[checkerboard_dim-1], s=200)
    plt.plot([1,2,3,4,5], y, c=sns.color_palette("colorblind")[checkerboard_dim-1], ls='--', lw=1.5)
plt.ylim([-0.04, 1.04])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xticks([1,2,3,4,5], ['100', '1000', '10 000', '100 000', '1 000 000'])
plt.xlabel('Number of points sampled to estimate the value function')
plt.ylabel('Fraction of checkerboard function\nthat is correctly estimated')
plt.savefig('../../figures/checkerboard_estimation.pdf')
plt.show()

### Plots for individual degrees

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 2},  font_scale=2)

for checkerboard_dim in range(2, 11):    
    plt.figure(figsize=(6, 6))
    plt.ylim([-0.04, 1.04])
    for interaction_order in range(1,11):
        y, y_min, y_max = [], [], [] 
        for num_samples in [100, 1000, 10000, 100000, 1000000]:
            yy = []
            for x in results[checkerboard_dim][num_samples]:
                order_sum =  np.sum([np.abs(v) for k, v in x.items() if len(k) == interaction_order]) 
                all_sum = np.sum([np.abs(v) for k, v in x.items()])
                yy.append(order_sum / all_sum)
            y.append(np.mean(yy))
            y_min.append(np.min(yy))
            y_max.append(np.max(yy))
        if np.max(y) > 0.01:
            sns.scatterplot(x=[1,2,3,4,5], y=y, color=sns.color_palette("tab10")[interaction_order-1], s=200)
            plt.plot([1,2,3,4,5], y, c=sns.color_palette("colorblind")[interaction_order-1], ls='--', lw=1.5)
            # convert y_min and y_max from y coordinates to plot range
            y_min = [x/1.08+0.04 for x in y_min]
            y_max = [x/1.08+0.04 for x in y_max]
            for x_pos in [1,2,3,4,5]:
                plt.axvline(x=x_pos, ymin=y_min[x_pos-1], ymax=y_max[x_pos-1], c=sns.color_palette("colorblind")[interaction_order-1], lw=4)
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.xticks([1,2,3,4,5], ['100', '1000', '10 000', '100 000', '1 000 000'], rotation=20)
    plt.title(f'{checkerboard_dim}d Checkerboard Function')
    plt.xlabel('Number of points sampled')
    plt.ylabel('Order of estimated effects')
    plt.tight_layout()
    plt.savefig(f'../../figures/{checkerboard_dim}_checkerboard_estimation.pdf')
    plt.show()